In [1]:
import os
import gc
import pickle

import numpy as np
import pandas as pd
import polars as pl
# import plotly.express as px
# from plotly.subplots import make_subplots

from sklearn.ensemble import VotingRegressor

import lightgbm as lgb
# from xgboost import XGBClassifier
# from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
root = "/kaggle/input/predict-energy-behavior-of-prosumers"

df_train = pd.read_csv(os.path.join(root, "train.csv"))
df_client = pd.read_csv(os.path.join(root, "client.csv"))
# df_gas_prices = pd.read_csv(os.path.join(root, "gas_prices.csv"))
# df_electricity_prices = pd.read_csv(os.path.join(root, "electricity_prices.csv"))
# df_forecast_weather = pd.read_csv(os.path.join(root, "forecast_weather.csv"))
# df_historical_weather = pd.read_csv(os.path.join(root, "historical_weather.csv"))
# df_weather_station_to_county_mapping = pd.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"))

In [4]:
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60


In [5]:
client_latest = df_client.groupby(['county','product_type','is_business']).max().reset_index()

In [6]:
client_latest.drop(['date','data_block_id'],axis=1,inplace=True)

In [7]:
client_latest.rename(columns = {'eic_count':'eic_count_latest','installed_capacity':'installed_capacity_latest'}, inplace = True) 

In [8]:
client_latest

,county,product_type,is_business,eic_count_latest,installed_capacity_latest
0,0,0,1,30,1273.200
1,0,1,0,547,5250.705
2,0,1,1,99,2900.600
3,0,2,0,31,220.500
4,0,2,1,23,288.700
5,0,3,0,1517,15980.560
6,0,3,1,482,19314.310
7,1,1,0,20,188.700
8,1,3,0,75,694.400
9,1,3,1,19,367.700


In [9]:
df_client

,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2
4,1,0,43,1411.00,1,2021-09-01,2
...,...,...,...,...,...,...,...
41914,1,15,51,415.60,0,2023-05-29,637
41915,3,15,161,2035.75,0,2023-05-29,637
41916,0,15,15,620.00,1,2023-05-29,637
41917,1,15,20,624.50,1,2023-05-29,637


In [10]:
# df_train[(df_train['data_block_id']==54)&(df_train['county']==0)&(df_train['is_business']==0)&(df_train['product_type']==1)&(df_train['is_consumption']==0)]

In [11]:
# df_forecast_weather[(df_forecast_weather['data_block_id']==54)&(df_forecast_weather['latitude']==57.6)&(df_forecast_weather['longitude']==21.7)]

# Read Data

In [12]:
root = "/kaggle/input/predict-energy-behavior-of-prosumers"

data_cols = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id']
client_cols = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
gas_prices_cols = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
electricity_prices_cols = ['forecast_date', 'euros_per_mwh']
forecast_weather_cols = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_weather_cols = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols = ['longitude', 'latitude', 'county']
target_cols = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']

In [13]:
df_data = pl.read_csv(os.path.join(root, "train.csv"), columns=data_cols, try_parse_dates=True)
df_client = pl.read_csv(os.path.join(root, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas_prices = pl.read_csv(os.path.join(root, "gas_prices.csv"), columns=gas_prices_cols, try_parse_dates=True)
df_electricity_prices = pl.read_csv(os.path.join(root, "electricity_prices.csv"), columns=electricity_prices_cols, try_parse_dates=True)
df_forecast_weather = pl.read_csv(os.path.join(root, "forecast_weather.csv"), columns=forecast_weather_cols, try_parse_dates=True)
df_historical_weather = pl.read_csv(os.path.join(root, "historical_weather.csv"), columns=historical_weather_cols, try_parse_dates=True)
df_weather_station_to_county_mapping = pl.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"), columns=location_cols, try_parse_dates=True)
df_target = df_data.select(target_cols)

schema_data = df_data.schema
schema_client = df_client.schema
schema_gas  = df_gas_prices.schema
schema_electricity = df_electricity_prices.schema
schema_forecast = df_forecast_weather.schema
schema_historical = df_historical_weather.schema
schema_target = df_target.schema

# Features Generation

Polars library is faster than Pandas, which explains the choice.

In [14]:
def generate_features(
        df_data, 
        df_client, 
        df_gas_prices, 
        df_electricity_prices, 
        df_forecast_weather, 
        df_historical_weather, 
        df_weather_station_to_county_mapping, 
        df_target
):
    df_data = (
        df_data
        .with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )
    )
    
    df_gas_prices = (
        df_gas_prices
        .rename({"forecast_date": "date"})
    )
    
    df_electricity_prices = (
        df_electricity_prices
        .rename({"forecast_date": "datetime"})
    )
    
    df_weather_station_to_county_mapping = (
        df_weather_station_to_county_mapping
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32)
        )
    )
    
    # sum of all product_type targets related to ["datetime", "county", "is_business", "is_consumption"]
    df_target_all_type_sum = (
        df_target
        .group_by(["datetime", "county", "is_business", "is_consumption"]).sum()
        .drop("product_type")
    )
    
    df_forecast_weather = (
        df_forecast_weather
        .rename({"forecast_datetime": "datetime"})
        .filter( (pl.col("hours_ahead") >= 22) & (pl.col("hours_ahead") <= 45) ) # we don't need forecast for today
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            # datetime for forecast in a different timezone
            pl.col('datetime').dt.replace_time_zone(None).cast(pl.Datetime("us"))
        )
        .join(df_weather_station_to_county_mapping, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_historical_weather = (
        df_historical_weather
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
#            pl.col("datetime") + pl.duration(hours=37)
        )
        .join(df_weather_station_to_county_mapping, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    # creating average forecast characteristics for all weather stations
    df_forecast_weather_date = (
        df_forecast_weather
        .group_by("datetime").mean()
        .drop("county")
    )
    
    # creating average forecast characteristics for weather stations related to county
    df_forecast_weather_local = (
        df_forecast_weather
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    
    # creating average historical characteristics for all weather stations
    df_historical_weather_date = (
        df_historical_weather
        .group_by("datetime").mean()
        .drop("county")
    )
    
    # creating average historical characteristics for weather stations related to county
    df_historical_weather_local = (
        df_historical_weather
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    
    df_data = (
        df_data
        # pl.duration(days=1) shifts datetime to join lag features (usually we join last available values)
        .join(df_gas_prices.with_columns((pl.col("date") + pl.duration(days=1)).cast(pl.Date)), on="date", how="left")
        .join(df_client.with_columns((pl.col("date") + pl.duration(days=2)).cast(pl.Date)), on=["county", "is_business", "product_type", "date"], how="left")
        .join(df_client.with_columns((pl.col("date") + pl.duration(days=7)).cast(pl.Date)), on=["county", "is_business", "product_type", "date"], how="left", suffix="_client_7d")
        .join(df_client.with_columns((pl.col("date")).cast(pl.Date)), on=["county", "is_business", "product_type", "date"], how="left", suffix="_0d")
        .join(df_electricity_prices.with_columns(pl.col("datetime") + pl.duration(days=1)), on="datetime", how="left")
        
        # lag forecast_weather features (24 hours * days)
        .join(df_forecast_weather_date, on="datetime", how="left", suffix="_fd")
        .join(df_forecast_weather_local, on=["county", "datetime"], how="left", suffix="_fl")
        .join(df_forecast_weather_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_fd_7d")
        .join(df_forecast_weather_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_fl_7d")

        # lag historical_weather features (24 hours * days)
        .join(df_historical_weather_date.with_columns(pl.col("datetime") + pl.duration(days=2)), on="datetime", how="left", suffix="_hd_2d")
        .join(df_historical_weather_local.with_columns(pl.col("datetime") + pl.duration(days=2)), on=["county", "datetime"], how="left", suffix="_hl_2d")
        .join(df_historical_weather_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_hd_7d")
        .join(df_historical_weather_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_hl_7d")
        
        # lag target features (24 hours * days)
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
#         .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
#         .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
#         .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
#         .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        
        
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"),
            pl.col("datetime").dt.hour().alias("hour"),
            pl.col("datetime").dt.day().alias("day"),
            pl.col("datetime").dt.weekday().alias("weekday"),
            pl.col("datetime").dt.month().alias("month"),
            pl.col("datetime").dt.year().alias("year"),
        )
        
        .with_columns(
            pl.concat_str("county", "is_business", "product_type", "is_consumption", separator="_").alias("segment"),
        )
        
        # cyclical features encoding https://towardsdatascience.com/cyclical-features-encoding-its-about-time-ce23581845ca
        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
        )
        
        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )
        
        .drop( 'date', "hour", "dayofyear")#datetime
#         .drop("date", "hour", "dayofyear")
    )
    
    return df_data

In [15]:
def to_pandas(X, y=None):
    cat_cols = ["county", "is_business", "product_type", "is_consumption", "segment"]
    
    if y is not None:
        df = pd.concat([X.to_pandas(), y.to_pandas()], axis=1)
    else:
        df = X.to_pandas()    
    
    df = df.set_index("row_id")
    df[cat_cols] = df[cat_cols].astype("category")
    
    df["target_mean1"] = df[[f"target_{i}" for i in [1,2] ]].mean(1)
    df["target_mean"] = df[[f"target_{i}" for i in [1,2,6,7] ]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in [1,2,6,7] ]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)
    
    return df

In [16]:
df_data, y = df_data.drop("target"), df_data.select("target")

df_train_features = generate_features(
    df_data, 
    df_client, 
    df_gas_prices, 
    df_electricity_prices, 
    df_forecast_weather, 
    df_historical_weather, 
    df_weather_station_to_county_mapping, 
    df_target
)

df_train_features = to_pandas(df_train_features, y)
# a little proportion of target values are null
df_train_features = df_train_features[df_train_features['target'].notnull()]

# filter old data
# df_train_features = df_train_features[df_train_features.year >= 2022]

In [17]:
# df_train_features[df_train_features['installed_capacity'].isnull()]

In [18]:
df_train_features['installed_capacity'].fillna(df_train_features['installed_capacity_0d'],inplace=True)
df_train_features['eic_count'].fillna(df_train_features['eic_count_0d'],inplace=True)
df_train_features['installed_capacity_client_7d'].fillna(df_train_features['installed_capacity_0d'],inplace=True)
df_train_features['eic_count_client_7d'].fillna(df_train_features['eic_count_0d'],inplace=True)
df_train_features.drop(['installed_capacity_0d','eic_count_0d'],axis=1,inplace=True)

In [19]:
# df_train_features['installed_capacity'].isnull().sum()

In [20]:
# df_train_features = df_train_features[df_train_features['installed_capacity'].notna()]


In [21]:
import holidays
import datetime

estonian_holidays = holidays.country_holidays('EE', years=range(2021, 2026))
estonian_holidays = list(estonian_holidays.keys())

def add_holidays_as_binary_features(df):
    df['country_holiday'] = df.apply(lambda row: (datetime.date(row['year'], row['month'], row['day']) in estonian_holidays) * 1, axis=1)
    
    df.drop(['year','day','segment'],axis=1,inplace=True)
    df.drop(['hours_ahead','hours_ahead_fl','hours_ahead_fd_7d','hours_ahead_fl_7d'],axis=1,inplace=True)
    df['working_hour'] = ((df['datetime'].dt.hour>=9)&(df['datetime'].dt.hour<=18))
    df.drop(['datetime'],axis=1,inplace=True)

    df['eic_count_increase_7d'] = df['eic_count'] - df['eic_count_client_7d']
    df['capacity_increase_7d'] = df['installed_capacity'] - df['installed_capacity_client_7d']

    df['weekend'] = ((df['weekday']==6)|(df['weekday']==7))
    return df

# df_train_features['installed_capacity'].fillna(value=1,inplace=True)
df_train_features = add_holidays_as_binary_features(df_train_features)

# Model

In [22]:
# df_train_features.drop(['year','day','segment'],axis=1,inplace=True)
# df_train_features.drop(['hours_ahead','hours_ahead_fl','hours_ahead_fd_7d','hours_ahead_fl_7d'],axis=1,inplace=True)

In [23]:
# df_train_features = df_train_features[df_train_features['installed_capacity'].notna()]

In [24]:
# ## "county", "is_business", "product_type", "is_consumption"
# product_type = 0
# is_business = 0
# is_consumption = 0
# county = 4
# segment = '12_1_3_1'
# df_plot = df_train_features[
#     (df_train_features['segment'] == segment)     
# #     (df_train_features['is_business'] == is_business)       &
# #     (df_train_features['is_consumption'] == is_consumption) &
# #     (df_train_features['county'] == county)
# ]

# fig = px.line(
#     df_plot, 
#     x="datetime", 
#     y="country_holiday", 
# #     color='county', 
# #     title=f'Target for county={county}, is_busines={is_business}'
# )
# fig.show()

# fig = px.line(
#     df_plot, 
#     x="datetime", 
#     y="target")
# fig.show()

In [25]:
# df_train_features['working_hour'] = ((df_train_features['datetime'].dt.hour>=9)&(df_train_features['datetime'].dt.hour<=18))
# df_train_features.drop(['datetime'],axis=1,inplace=True)

# df_train_features['eic_count_increase_7d'] = df_train_features['eic_count'] - df_train_features['eic_count_client_7d']
# df_train_features['capacity_increase_7d'] = df_train_features['installed_capacity'] - df_train_features['installed_capacity_client_7d']

# df_train_features['weekend'] = ((df_train_features['weekday']==6)|(df_train_features['weekday']==7))

In [26]:
# list(df_train_features.columns) 

In [27]:
train_size = int(0.85 * len(df_train_features))
test_size = len(df_train_features) - train_size

df_train_features_train = df_train_features#.iloc[:train_size]
# df_train_features_test = df_train_features.iloc[train_size:]

# train_x2, val_x2, train_y2, val_y2 = train_test_split(df_train_features_train.drop(columns=["target"]), df_train_features_train['target'], test_size=0.3, random_state=42)

In [28]:
# Let's build separate models for produciton and.aggregateconsumption of electricity

model_parameters1 = {
    'n_estimators': 1800,
    'learning_rate': 0.04,
    'colsample_bytree': 0.8,
    'colsample_bynode': 0.6,
    'lambda_l1': 3.5,
    'lambda_l2': 1.5,
    'max_depth': 12,
    'device': 'gpu',
    'subsample' : 0.5,    
    'objective': "regression_l1",
#     "early_stopping_rounds":50 , # only if eval set is there
}

model_parameters0 = {
    'n_estimators': 1200,
    'learning_rate': 0.04,
    'colsample_bytree': 0.7,
    'colsample_bynode': 0.6,
    'lambda_l1': 3.5,
    'lambda_l2': 1.5,
    'max_depth': 8,
    'device': 'gpu',
    'subsample' : 0.5,    
    'objective': "regression_l1",
#     "early_stopping_rounds":50 ,  # only if eval set is there
}

# ##standard technique to use mean predictions from several models with different random_state
model_consumption = VotingRegressor([
    (
        f'consumption_lgb_{i}', 
         lgb.LGBMRegressor(**model_parameters1, random_state=i)
    ) for i in range(2)
])
model_production = VotingRegressor([
    (
        f'production_lgb_{i}', 
         lgb.LGBMRegressor(**model_parameters0, random_state=i)
    ) for i in range(2)
])

mask = df_train_features_train['is_consumption'] == 1
model_consumption.fit(
    X=df_train_features_train[mask].drop(columns=["target"]),
    y=df_train_features_train[mask]["target"]
)

mask = df_train_features_train['is_consumption'] == 0
model_production.fit(
    X=df_train_features_train[mask].drop(columns=["target"]),
    y=df_train_features_train[mask]["target"]/df_train_features_train[mask]["installed_capacity"]
)

# model_consumption = lgb.LGBMRegressor(**model_parameters1, random_state=99)
# model_production = lgb.LGBMRegressor(**model_parameters0, random_state=99)

# mask = df_train_features_train['is_consumption'] == 0
# train_x2, val_x2, train_y2, val_y2 = train_test_split(df_train_features_train[mask].drop(columns=["target"]), df_train_features_train[mask]['target']/df_train_features_train[mask]["installed_capacity"], test_size=0.3, random_state=42)


# mask = df_train_features_train['is_consumption'] == 1
# train_x2, val_x2, train_y2, val_y2 = train_test_split(df_train_features_train[mask].drop(columns=["target"]), df_train_features_train[mask]['target'], test_size=0.3, random_state=42)

# model_consumption.fit(
#     X=train_x2,
#     y=train_y2,
#     eval_set = [(val_x2,val_y2)]
# )

# mask = df_train_features_train['is_consumption'] == 0
# train_x2, val_x2, train_y2, val_y2 = train_test_split(df_train_features_train[mask].drop(columns=["target"]), df_train_features_train[mask]['target']/df_train_features_train[mask]["installed_capacity"], test_size=0.3, random_state=42)
# model_production.fit(
#     X=train_x2,
#     y=train_y2,
#     eval_set = [(val_x2,val_y2)]
# )


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


[LightGBM] [Warning] lambda_l1 is set=3.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.5
[LightGBM] [Warning] lambda_l2 is set=1.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: Futu

VotingRegressor(estimators=[('production_lgb_0',
                             LGBMRegressor(colsample_bynode=0.6,
                                           colsample_bytree=0.7, device='gpu',
                                           lambda_l1=3.5, lambda_l2=1.5,
                                           learning_rate=0.04, max_depth=8,
                                           n_estimators=1200,
                                           objective='regression_l1',
                                           random_state=0, subsample=0.5)),
                            ('production_lgb_1',
                             LGBMRegressor(colsample_bynode=0.6,
                                           colsample_bytree=0.7, device='gpu',
                                           lambda_l1=3.5, lambda_l2=1.5,
                                           learning_rate=0.04, max_depth=8,
                                           n_estimators=1200,
                                           objective='regression_l1',
                                           random_state=1, subsample=0.5))])

In [29]:
# mask = df_train_features_test['is_consumption'] == 1
# y_pred1 = model_consumption.predict(
    
#     df_train_features_test[mask].drop(columns=["target"])
# )

# mask = df_train_features_test['is_consumption'] == 0
# y_pred0 = model_production.predict(
#     df_train_features_test[mask].drop(columns=["target"])
# )

In [30]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# def plotImp(model,num = 60, fig_size = (40, 50)):
#     feature_imp = pd.DataFrame({'Value':model.feature_importances_,'Feature':model.feature_name_})
#     plt.figure(figsize=fig_size)
#     sns.set(font_scale = 5)
#     sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
#                                                         ascending=False)[0:num])
#     plt.title('LightGBM Features (avg over folds)')
#     plt.tight_layout()
#     plt.savefig('lgbm_importances-01.png')
#     plt.show()

In [31]:
# plotImp(model_production)

In [32]:
# plotImp(model_consumption)

In [33]:
# mask = df_train_features_test['is_consumption'] == 1
# mean_absolute_error(df_train_features_test[mask]["target"], y_pred1)

In [34]:
# mask = df_train_features_test['is_consumption'] == 0
# mean_absolute_error(df_train_features_test[mask]["target"], (y_pred0*df_train_features_test[mask]['installed_capacity']))

In [35]:
# import pickle
# with open('model_consumption.pkl','wb') as f:
#     pickle.dump(model_consumption,f)
# with open('model_production.pkl','wb') as f:
#     pickle.dump(model_production,f)


In [36]:
# with open('model_consumption.pkl', 'rb') as f:
#     model_consumption = pickle.load(f)
# with open('model_production.pkl', 'rb') as f:
#     model_production = pickle.load(f)

In [37]:
# mask = df_train_features['is_consumption'] == 0
# model_production.predict(df_train_features[mask].drop(columns=["target"])).clip(0)*df_train_features[mask]['installed_capacity'].values/100

In [38]:
# mask = df_train_features['is_consumption'] == 0
# model_production.predict(df_train_features[mask].drop(columns=["target"])).clip(0)*df_train_features[mask]['installed_capacity'].values/100

# Submit API

In [39]:
client_latest.head()

,county,product_type,is_business,eic_count_latest,installed_capacity_latest
0,0,0,1,30,1273.200
1,0,1,0,547,5250.705
2,0,1,1,99,2900.600
3,0,2,0,31,220.500
4,0,2,1,23,288.700


In [40]:
# Function to fill null values using lookup table and replace missing values with mean
def fill_null_with_lookup(df, lookup_table):
    merged_df = pd.merge(df, lookup_table, on=['county', 'product_type', 'is_business'], how='left')
    
    # Fill null values with lookup table values
    df['installed_capacity'] = df['installed_capacity'].combine_first(merged_df['installed_capacity_latest'])
    df['eic_count'] = df['eic_count'].combine_first(merged_df['eic_count_latest'])
    df['installed_capacity_client_7d'] = df['installed_capacity_client_7d'].combine_first(merged_df['installed_capacity_latest'])
    df['eic_count_client_7d'] = df['eic_count_client_7d'].combine_first(merged_df['eic_count_latest'])
    
    # Replace remaining null values with mean
    df['installed_capacity'].fillna(df['installed_capacity'].mean(), inplace=True)
    df['eic_count'].fillna(df['eic_count'].mean(), inplace=True)
    df['installed_capacity_client_7d'].fillna(df['installed_capacity_client_7d'].mean(), inplace=True)
    df['eic_count_client_7d'].fillna(df['eic_count_client_7d'].mean(), inplace=True)


# Applying the function
# fill_null_with_lookup(df_test_features, client_latest)


In [41]:
# model_consumption = lgb.Booster(model_file='model_consumption.txt')
# model_production = lgb.Booster(model_file='model_production.txt')
import enefit

env = enefit.make_env()
iter_test = env.iter_test()

The following code demonstrated usage of API when in each cycle step we simulate new day and we need to send prediciotns before we get the next day (this guranteeres that we don't see targets from future).

Local running of a notebook uses toy example of test data, after notebook submission toy data will be replaced with new unseen data.

In [42]:
for (
    test, 
    revealed_targets, 
    client, 
    historical_weather,
    forecast_weather, 
    electricity_prices, 
    gas_prices, 
    sample_prediction
) in iter_test:
#     print(df_client)
    
    test = test.rename(columns={"prediction_datetime": "datetime"})
    
    df_test = pl.from_pandas(test[data_cols[1:]], schema_overrides=schema_data)
    df_client = pl.from_pandas(client[client_cols], schema_overrides=schema_client)
    df_gas_prices = pl.from_pandas(gas_prices[gas_prices_cols], schema_overrides=schema_gas)
    df_electricity_prices = pl.from_pandas(electricity_prices[electricity_prices_cols], schema_overrides=schema_electricity)
    df_new_forecast_weather = pl.from_pandas(forecast_weather[forecast_weather_cols], schema_overrides=schema_forecast)
    df_new_historical_weather = pl.from_pandas(historical_weather[historical_weather_cols], schema_overrides=schema_historical)
    df_new_target = pl.from_pandas(revealed_targets[target_cols], schema_overrides=schema_target)
    
    df_forecast_weather = pl.concat([df_forecast_weather, df_new_forecast_weather]).unique(['forecast_datetime', 'latitude', 'longitude', 'hours_ahead'])
    df_historical_weather = pl.concat([df_historical_weather, df_new_historical_weather]).unique(['datetime', 'latitude', 'longitude'])
    df_target = pl.concat([df_target, df_new_target]).unique(['datetime', 'county', 'is_business', 'product_type', 'is_consumption'])
    
    df_test_features = generate_features(
        df_test, 
        df_client, 
        df_gas_prices, 
        df_electricity_prices, 
        df_forecast_weather, 
        df_historical_weather, 
        df_weather_station_to_county_mapping, 
        df_target
    )
    df_test_features = to_pandas(df_test_features)
    df_test_features.drop(['installed_capacity_0d','eic_count_0d'],axis=1,inplace=True)
    fill_null_with_lookup(df_test_features, client_latest)
    
    df_test_features = add_holidays_as_binary_features(df_test_features)
    
#     val = max(1,df_test_features['installed_capacity'].mean())
    
    
    mask = df_test_features['is_consumption'] == 1
    # clip method makes values < 0 equal 0 because our target is nonnegative and models can produce negative values
    sample_prediction.loc[mask.values, "target"] = model_consumption.predict(df_test_features[mask]).clip(0)
    
    mask = df_test_features['is_consumption'] == 0
    sample_prediction.loc[mask.values, "target"] = model_production.predict(df_test_features[mask]).clip(0)*df_test_features[mask]['installed_capacity'].values
    
#     print(sample_prediction)
    # send predictions
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
